In [ ]:
pip install langchain langchain_community langchain_chroma 

In [2]:
pip install -qU langchain-mistralai


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install requests

In [8]:
import getpass
import os
    
os.environ["MISTRAL_API_KEY"] = getpass.getpass()

from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model="open-mistral-7b")

In [ ]:
import bs4
import os

from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader,SitemapLoader,TextLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents import Document
from langchain_mistralai import MistralAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

import requests

# response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
# text = response.text
# f = open('essay.txt', 'w')
# f.write(text)
# f.close()

# loader = TextLoader("essay.txt")
# docs = loader.load()
# # Split text into chunks 
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,
#     chunk_overlap=20,
#     length_function=len,
#     is_separator_regex=False,)
# documents = text_splitter.split_documents(docs)

# print("documents", documents[0])

mistral = MistralAIEmbeddings(
    model="mistral-embed",
    mistral_api_key=os.environ["MISTRAL_API_KEY"]
)
try:
    document_sample = Document(page_content="Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep",
                               metadata={"source": "https://example.com"}
)
    vectorstore = Chroma.from_documents(documents=[document_sample], embedding=mistral)
 # Retrieve and generate using the relevant snippets of the blog.
   
except Exception as e:
   raise e






In [47]:
retriever = vectorstore.as_retriever()
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:
    <context>
    {context}
    </context>
    Question: {input}""")


document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "What were the two main things the author worked on before college?"})
print(response["answer"])


Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


HTTPStatusError: Error response 429 while fetching https://api.mistral.ai/v1/chat/completions: {"message":"Requests rate limit exceeded"}

In [46]:
retriever = vectorstore.as_retriever()
prompt = ChatPromptTemplate.from_template("""Generate further questions based on the question and only on the provided context:
    <context>
    {context}
    </context>
    Question: {input}""")


document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "What were the two main things the author worked on before college?"})
print(response["answer"])

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


HTTPStatusError: Error response 429 while fetching https://api.mistral.ai/v1/chat/completions: {"message":"Requests rate limit exceeded"}